In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load .env file from the same directory as this notebook
# In Jupyter, the current working directory is typically where the notebook is located
try:
    env_file = Path('.env')
    if env_file.exists():
        load_dotenv(dotenv_path=env_file, override=True)
        google_api_key = os.getenv("GOOGLE_API_KEY")
        if not google_api_key:
            raise ValueError("GOOGLE_API_KEY not found in .env file")
        print(f"✅ Gemini API key loaded from .env file (first 10 chars: {google_api_key[:10]}...)")
        print(f"✅ Verified: GOOGLE_API_KEY is set in environment (length: {len(google_api_key)} chars)")
        print("ℹ️  ADK will use GOOGLE_API_KEY")
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your .env file. Details: {e}")

✅ Gemini API key loaded from .env file (first 10 chars: AIzaSyCxBj...)
✅ Verified: GOOGLE_API_KEY is set in environment (length: 39 chars)
ℹ️  ADK will use GOOGLE_API_KEY


In [2]:
import uuid
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.runners import InMemoryRunner
from google.adk.sessions import InMemorySessionService

from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.tool_context import ToolContext
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig
from google.adk.tools.function_tool import FunctionTool

print("Google imports happened")

Google imports happened


In [3]:
retry_config = types.HttpRetryOptions(
    attempts=5,  
    exp_base=7,  
    initial_delay=1,
    http_status_codes=[429, 500, 503, 504],
)

In [4]:
# Point to your custom Python MCP server
server_params = StdioServerParameters(
    command="python",
    args=["/Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/Day 2/examples/travel_mcp_server/server.py"],
    tool_filter=["getWeatherForecast", "searchFlights"],
)

# Create MCP toolset
travel_mcp_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=server_params,
        timeout=30,
    )
)

# Create agent with your MCP tools (now has both weather AND flight tools)
travel_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="travel_agent",
    instruction="""You are a comprehensive travel planning assistant. 
    
    You have access to:
    - Weather forecast tool: Get weather and packing suggestions for destinations
    - Flight search tool: Find flight options between airports
    
    When helping users plan trips:
    1. Search for flights if they mention travel between cities
    2. Check weather at the destination
    3. Provide packing suggestions based on weather
    4. Combine all information into a helpful travel plan
    """,
    tools=[travel_mcp_server],
)

# Create runner
runner = InMemoryRunner(agent=travel_agent)

# Test both tools - flights and weather
response = await runner.run_debug(
    "I want to travel from Mumbai (BOM) to London (LHR) on December 15, 2025. "
    "Can you find me flight options and tell me what's the weather like in London "
    "from December 15 to December 22? What should I pack?",
    verbose=True
)


 ### Created new session: debug_session_id

User > I want to travel from Mumbai (BOM) to London (LHR) on December 15, 2025. Can you find me flight options and tell me what's the weather like in London from December 15 to December 22? What should I pack?


/Users/ayoushchourasia/Documents/learning/5-Day-AI-Agents-Intensive-Course-with-Google/env/lib/python3.13/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:101: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(
Both GOOGLE_API_KEY and GEMINI_API_KEY are set. Using GOOGLE_API_KEY.


travel_agent > [Calling tool: search_flights({'departure_date': '2025-12-15', 'destination': 'L...)]
travel_agent > [Calling tool: get_weather_forecast({'destination': 'London', 'travel_dates': '2025-12...)]
travel_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "status": "success",\n  "route": {\n    "origin": {\n   ...]
travel_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "status": "success",\n  "destination": "London",\n  "cou...]
travel_agent > Here are your flight options from Mumbai to London for December 15, 2025:

*   **United Airlines:** Departs BOM at 07:00, arrives LHR at 15:00. Duration: 8 hours. Price: $619.
*   **Delta Air Lines:** Departs BOM at 10:00, arrives LHR at 18:00. Duration: 8 hours. Price: $678.
*   **United Airlines:** Departs BOM at 14:45, arrives LHR at 22:45. Duration: 8 hours. Price: $832.

The weather in London from December 15 to December 22 is expected to be between 1.6°C and 14.4°C (35°F to 58°F). The conditions wi